In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from file_utils import load_dataframes
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt

sns.set_theme()

dataset_dir = 'data'

In [2]:
items = pd.read_csv(f'{dataset_dir}/items.csv', na_values='unknown',
                    dtype={'item_id': 'Int64', 'alley_id': 'Int64', 'category': 'Int64'})
items.drop(columns='Unnamed: 0', inplace=True)
items.dropna(subset=['item_id'], inplace=True)

In [3]:
categories = pd.read_csv('data/categories.csv', index_col='category_id')
categories.drop(columns='Unnamed: 0', inplace=True)

In [4]:
alley_inventory = pd.read_csv('data/alley_inventory.csv', index_col='alley_id')

In [5]:
transaction_items__prior = pd.read_csv('data/transaction_items__prior.csv', na_values='unknown',
                                       dtype={'transaction_id': 'Int64', 'item_id': 'Int64',
                                              'add_to_cart_order': 'Int16', 'previous_bought': 'Int8'})
transaction_items__prior.drop(columns=['Unnamed: 0'], inplace=True)
transaction_items__prior.dropna(subset=['transaction_id', 'item_id'], inplace=True)

In [6]:
transactions = pd.read_csv('data/transactions.csv', na_values='unknown',
                           dtype={'transaction_id': 'Int64', 'customer_id': 'Int64', 'transaction_number': 'Int64',
                                  'day_of_week': 'category', 'time_of_day': 'Int8',
                                  'days_since_prior_order': 'Int64'})

transactions.drop(columns='Unnamed: 0', inplace=True)
transactions.dropna(subset=['transaction_id'], inplace=True)
transactions.set_index(['transaction_id'], inplace=True)

In [7]:
items_data = pd.merge(items, categories, left_on='category', right_on='category_id', how='left')
items_data = pd.merge(items_data, alley_inventory, left_on='alley_id', right_on='alley_id', how='left')
items_data.drop(columns=['category_x', 'alley_id'], inplace=True)
items_data.rename(columns={'category_y': 'category'}, inplace=True)

In [8]:
transaction_items = pd.merge(transaction_items__prior, items_data, on='item_id')
transaction_items['quantity'] = 1

In [9]:
transactions__prior = transactions[transactions['eval_set'] == 'prior']
transaction_items = pd.merge(transaction_items, transactions__prior, on='transaction_id')

In [10]:
transaction_items.head()

,transaction_id,item_id,add_to_cart_order,previous_bought,item_name,category,alley,quantity,customer_id,eval_set,transaction_number,day_of_week,time_of_day,days_since_prior_order
0,2,33120,1,1,Organic Egg Whites,dairy eggs,eggs,1,202279,prior,3,5.0,9,8
1,2,45918,4,1,Coconut Butter,pantry,oils vinegars,1,202279,prior,3,5.0,9,8
2,2,30035,5,0,Natural Sweetener,pantry,baking ingredients,1,202279,prior,3,5.0,9,8
3,2,17794,6,1,Carrots,produce,fresh vegetables,1,202279,prior,3,5.0,9,8
4,2,40141,7,1,Original Unflavored Gelatine Mix,pantry,doughs gelatins bake mixes,1,202279,prior,3,5.0,9,8


In [11]:
total_bought = transaction_items.groupby('item_id')['quantity'].count().sort_values(ascending=False)
items_data = pd.merge(items_data, total_bought, on='item_id')

In [12]:
items_data.head()

,item_id,item_name,category,alley,quantity
0,1,Chocolate Sandwich Cookies,snacks,cookies cakes,1579
1,2,All-Seasons Salt,pantry,spices seasonings,76
2,3,Robust Golden Unsweetened Oolong Tea,beverages,tea,229
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,frozen,frozen meals,279
4,5,Green Chile Anytime Sauce,pantry,marinades meat preparation,13


In [41]:
top_categories = items_data.groupby('category')['quantity'].sum().sort_values(ascending=False).head(10)

In [42]:
top_categories

category
produce            7740512
dairy eggs         4326411
snacks             2321773
beverages          2137287
frozen             1813659
pantry             1475197
bakery              935578
canned goods        868994
deli                863204
dry goods pasta     683225
Name: quantity, dtype: int64

In [43]:
top_categories_items = items_data[items_data['category'].isin(top_categories.index)]
grouped_items = top_categories_items.groupby('category')
top_items = []

for category, group in grouped_items:
    top_items.append(group.sort_values(by='quantity', ascending=False).head(10))

In [44]:
top_items = pd.concat(top_items)
top_items.set_index('item_id', inplace=True)

In [45]:
top_items

,item_name,category,alley,quantity
item_id,,,,
5077,100% Whole Wheat Bread,bakery,bread,52219
18362,Organic Bread with 21 Whole Grains,bakery,NaN,19755
18027,Ezekiel 4:9 Bread Organic Sprouted Whole Grain,bakery,bread,15277
21267,Sourdough Bread,bakery,bread,14064
10673,Original Nooks & Crannies English Muffins,bakery,breakfast bakery,12898
...,...,...,...,...
40199,Chocolate Chip Cookies,snacks,cookies cakes,12164
35561,NaN,snacks,chips pretzels,11832
3599,Baked Aged White Cheddar Rice and Corn Puffs,snacks,popcorn jerky,11735


In [46]:
transaction_top_items = transaction_items[transaction_items['item_id'].isin(top_items.index)]

In [47]:
transaction_top_items.head()

,transaction_id,item_id,add_to_cart_order,previous_bought,item_name,category,alley,quantity,customer_id,eval_set,transaction_number,day_of_week,time_of_day,days_since_prior_order
21,327,19660,1,1,Spring Water,beverages,water seltzer sparkling water,1,58707,prior,21,6.0,9,8
31,390,19660,1,1,Spring Water,beverages,water seltzer sparkling water,1,166654,prior,48,0.0,12,9
41,390,3957,4,1,100% Raw Coconut Water,beverages,refrigerated,1,166654,prior,48,0.0,12,9
61,537,21137,3,<NA>,Organic Strawberries,produce,fresh fruits,1,180135,prior,15,2.0,8,3
62,537,9076,4,1,Blueberries,frozen,frozen produce,1,180135,prior,15,2.0,8,3


In [48]:
print(f'Number of transactions: {transaction_top_items.shape[0]}')

Number of transactions: 4079103


In [49]:
def true_if_any_present(x):
    return x.any()

In [50]:
transactions_data = pd.pivot_table(
    transaction_top_items,
    index="transaction_id",
    columns="item_name",
    values="quantity",
    aggfunc=true_if_any_present,
    fill_value=False,
)

In [51]:
transactions_data.head()

item_name,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat Milk,Bag of Organic Bananas,Baked Aged White Cheddar Rice and Corn Puffs,Banana,Black Beans,Blueberries,Chicken & Maple Breakfast Sausage,Chocolate Chip Cookies,...,Sparkling Lemon Water,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Tomato Ketchup,Tomato Paste,Uncured Genoa Salami,Uncured Slow Cooked Ham,Unsweetened Almondmilk
transaction_id,,,,,,,,,,,,,,,,,,,,,
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [52]:
frequent_itemsets = apriori(transactions_data, min_support=0.01, use_colnames=True)

In [53]:
frequent_itemsets

,support,itemsets
0,0.017449,(100% Raw Coconut Water)
1,0.027860,(100% Whole Wheat Bread)
2,0.017003,(2% Reduced Fat Milk)
3,0.173919,(Bag of Organic Bananas)
4,0.216696,(Banana)
...,...,...
64,0.016279,"(Organic Baby Spinach, Organic Strawberries)"
65,0.010900,"(Organic Hass Avocado, Organic Raspberries)"
66,0.017363,"(Organic Strawberries, Organic Hass Avocado)"
67,0.014383,"(Organic Strawberries, Organic Raspberries)"


In [54]:
rules_mlxtend = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.03)

In [55]:
rules_mlxtend

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Organic Baby Spinach),(Bag of Organic Bananas),0.111050,0.173919,0.021406,0.192764,1.108351,0.002093,1.023344,0.109971
1,(Bag of Organic Bananas),(Organic Baby Spinach),0.173919,0.111050,0.021406,0.123082,1.108351,0.002093,1.013721,0.118340
2,(Organic Hass Avocado),(Bag of Organic Bananas),0.097937,0.173919,0.026514,0.270730,1.556641,0.009481,1.132750,0.396415
3,(Bag of Organic Bananas),(Organic Hass Avocado),0.173919,0.097937,0.026514,0.152452,1.556641,0.009481,1.064321,0.432877
4,(Bag of Organic Bananas),(Organic Raspberries),0.173919,0.062756,0.017117,0.098418,1.568259,0.006202,1.039555,0.438638
5,(Organic Raspberries),(Bag of Organic Bananas),0.062756,0.173919,0.017117,0.272750,1.568259,0.006202,1.135897,0.386612
6,(Organic Strawberries),(Bag of Organic Bananas),0.121534,0.173919,0.026190,0.215499,1.239076,0.005053,1.053002,0.219641
7,(Bag of Organic Bananas),(Organic Strawberries),0.173919,0.121534,0.026190,0.150590,1.239076,0.005053,1.034207,0.233569
8,(Organic Whole Milk),(Bag of Organic Bananas),0.063175,0.173919,0.011298,0.178836,1.028271,0.000311,1.005988,0.029348
9,(Bag of Organic Bananas),(Organic Whole Milk),0.173919,0.063175,0.011298,0.064961,1.028271,0.000311,1.001910,0.033283
